In [ ]:
!pip install -q pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.3 MB/s eta 0:00:00


In [ ]:
!wget -N -q "https://github.com/thomasfork/ipopt_linux/raw/main/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

In [ ]:
import numpy as np
from pyomo.environ import *

In [ ]:
def rosembrock(model):
  return (1 - model.x[1])**2 + 100. * (model.x[2] - model.x[1]**2)**2

def constraint1(model):
  return (model.x[1] - 1.)**3 - model.x[2] + 1 <= 0.

def constraint2(model):
  return model.x[1] + model.x[2] - 2. <= 0.

In [ ]:
# --- Create a model
model             = ConcreteModel()

model.x           = Var((1, 2), within = Reals)

model.x[1].value  = 3.
model.x[2].value  = -4.

model.x[1].lower  = -1.5
model.x[1].upper  = 1.5
model.x[2].lower  = -0.5
model.x[2].upper  = 2.5

# --- Declare objective function
model.objfun      = Objective(rule = rosembrock, sense = minimize)

model.Constraint1 = Constraint(rule = constraint1)
model.Constraint2 = Constraint(rule = constraint2)

model.pprint()

1 Var Declarations
    x : Size=2, Index={1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  -1.5 :   3.0 :   1.5 : False : False :  Reals
          2 :  -0.5 :  -4.0 :   2.5 : False : False :  Reals

1 Objective Declarations
    objfun : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : (1 - x[1])**2 + 100.0*(x[2] - x[1]**2)**2

2 Constraint Declarations
    Constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body                       : Upper : Active
        None :  -Inf : (x[1] - 1.0)**3 - x[2] + 1 :   0.0 :   True
    Constraint2 : Size=1, Index=None, Active=True
        Key  : Lower : Body              : Upper : Active
        None :  -Inf : x[1] + x[2] - 2.0 :   0.0 :   True

4 Declarations: x objfun Constraint1 Constraint2


In [ ]:
SolverFactory('ipopt', executable='/content/ipopt').solve(model).write()
#SolverFactory('multistart').solve(model, strategy = 'rand_guess_and_bound', solver = 'ipopt', iterations = 50).write()

# display solution
print('\nObjective function = ', model.objfun)

print('\nDecision Variables')
for i in model.x:
  print(model.x[i]())


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.015726327896118164
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objec